In [1]:
import logging
import ast
import sqlite3
import pandas as pd
import numpy as np

def load_sqlite_db_to_dataframe(db_path:str) -> pd.DataFrame:
    logging.basicConfig(filename='logs.log',
                        level=logging.INFO,
                        format='%(asctime)s:%(levelname)s:%(message)s')
    try:
        conn = sqlite3.connect(db_path)
        logging.info(f'Established SQLite connection with: {db_path}')
        query = "SELECT * FROM computer_organization_and_design_table"
        df = pd.read_sql_query(query, conn)
        df['embeddings'] = df['embeddings'].apply(ast.literal_eval)
        df['embeddings'] = np.stack(df['embeddings'].to_numpy()).tolist()
        logging.info('Database loaded and embeddings converted')
    except Exception as e:
        logging.error(f"Error loading database: {e}")
        raise e
    finally:
        conn.close()
        logging.info('Database connection closed')
    return df

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarities(df_embeddings, single_embedding):
    # Ensure df_embeddings is a 2D numpy array
    embeddings_array = np.array(df_embeddings.tolist())

    single_embedding_reshaped = np.reshape(single_embedding, (1, -1))
    print(single_embedding_reshaped.shape)

    # Calculate cosine similarity and directly convert to pandas Series.
    similarities = cosine_similarity(embeddings_array, single_embedding_reshaped).flatten()
    return pd.Series(similarities)

In [3]:
from openai import OpenAI

client = OpenAI()

def embed_text(text):
    response = client.embeddings.create(
        input = text,
        model = "text-embedding-3-large"
    )
    print("Embedding created.")
    return response.data[0].embedding

In [4]:
import pandas as pd

def search_by_embeddings(dataframe, user_question, n=5):
    embeded_user_q = embed_text(user_question)
    print("embeded_user_q created.")
    
    similarity_df = calculate_cosine_similarities(dataframe["embeddings"], embeded_user_q)
    print("similarity_df created.")
    
    dataframe['similarities'] = similarity_df
    print("dataframe['similarities'] created.")
    print(dataframe.head(5))
    
    res = dataframe.sort_values('similarities', ascending=False).head(n)
    print(res.head(5))
    return res


In [5]:
import pandas as pd

df = load_sqlite_db_to_dataframe('/Users/jdo/EnBed/EnBed/db/EnBed.sqlite')  # Provide a clear path as an argument
search_query = """Write a program in MIPS assembly language to convert an ASCII
                    number string containing positive and negative integer decimal strings, to an
                    integer. Your program should expect register $a0 to hold the address of a nullterminated
                    string containing some combination of the digits 0 through 9. Your
                    program should compute the integer value equivalent to this string of digits, then
                    place the number in register $v0. If a non-digit character appears anywhere in the
                    string, your program should stop with the value −1 in register $v0. For example,
                    if register $a0 points to a sequence of three bytes 50ten, 52ten, 0ten (the nullterminated
                    string “24”), then when the program stops, register $v0 should contain
                    the value 24ten.
                """
results = search_by_embeddings(df, search_query)
print(results)
print("Done")

Embedding created.
embeded_user_q created.
(1, 3072)
similarity_df created.
dataframe['similarities'] created.
   id                                  guid  \
0   1  a9084a5b-9900-48bb-bc17-69dc6a749d90   
1   2  214be393-5d82-440e-93e0-bda330cd38b7   
2   3  309cedf6-56ae-4f27-bad1-fa8ca679d333   
3   4  b1b7bee4-58e5-47f8-a499-38e9badfc362   
4   5  13ee0133-cd22-4ca8-a554-a1b5f1686290   

                                    author  \
0  David A. Patterson and John L. Hennessy   
1  David A. Patterson and John L. Hennessy   
2  David A. Patterson and John L. Hennessy   
3  David A. Patterson and John L. Hennessy   
4  David A. Patterson and John L. Hennessy   

                                               title  target_page page_range  \
0  Computer Organization and Design: The Hardware...            1      0,1,2   
1  Computer Organization and Design: The Hardware...            2      1,2,3   
2  Computer Organization and Design: The Hardware...            3      2,3,4   
3  Comput

In [6]:
# Assuming 'results' is the DataFrame with the relevant rows already selected
texts = results['text']
print(texts)

for i, text in enumerate(texts, 1):
    authority_info = f"Text {i}: {text}\n"
    print(authority_info)


192    <<<Page 170>>>\nChapter 2 Instructions: Langua...
127    Figure 2.14 summarizes the register convention...
95     2.4 Signed and Unsigned Numbers\n<<<Page 73>>>...
97     It’s up to the programming language, the opera...
138    <<<Page 116>>>\nChapter 2 Instructions: Langua...
Name: text, dtype: object
Text 1: <<<Page 170>>>
Chapter 2 Instructions: Language of the Computer
Consider the following MIPS loop:
```
LOOP: slt $t2, $0, $t1
beq $t2, $0, DONE
subi $t1, $t1, 1
addi $s2, $s2, 2
j LOOP
DONE:
```
Assume that the register $t1 is initialized to the value 10. What is the value in register $s2 assuming $s2 is initially zero? For each of the loops above, write the equivalent C code routine. Assume that the registers $s1, $s2, $t1, and $t2 are integers A, B, i, and temp, respectively. For the loops written in MIPS assembly above, assume that the register $t1 is initialized to the value N. How many MIPS instructions are executed? Translate the following C code to MIPS assembly code

In [7]:
import os
import openai


client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
def question_master(question, augmented_knowledge):
    system_prompt_static = """You are a helpful assistant that answers questions in computer science and software engineering. 
    You always think step-by-step and think for a moment before answering.
    You have the following tasks:
    1. Process and understand the information contained in the 'augmented_knowledge' variable.
        The 'augmented_knowledge' variable is enclosed between the XML tags "<augmented_knowledge>" and "</augmented_knowledge>", 
        and contains the most relevant information from the course textbook.
    2. Consider information derived from the 'augmented_knowledge' variable as completely true and fully accurate.
    3. Read the user's question.
    4. Take the information derived from the 'augmented_knwoledge' and use it to answer the user's question.
    5. Answer the user's question to the best of your ability, unless you couldn't find the answer in 'augmented_knowledge' 
        in which case say "I couldn't find it, would you like me to look it up?".
    Deliverables: 
    1. The most important information that you found in the 'augmented_knowledge' variable.
    2. The answer to the user's question, which may include generating code or a detailed explanation. Many times, it will include answering a multiple choice question.
    """;
    system_prompt_dynamic = "Here is the most relevant information drawn from the course textbook: \n\n" + augmented_knowledge
    system_prompt = system_prompt_static + system_prompt_dynamic
    response = client.chat.completions.create(
        # model="gpt-4",
        model="gpt-4-1106-preview",
        # model="gpt-3.5-turbo-0125",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    string_response = str(response.choices[0].message.content)
    return string_response


In [8]:
user_question = f'{search_query}'

In [9]:
answer = question_master(user_question, authority_info)
print(answer)

To write a MIPS assembly program that converts an ASCII number string to an integer, we need to consider the following steps:

1. Initialize the result to 0.
2. Loop through each character in the string.
3. Check if the character is a digit (between '0' and '9').
4. If it is a digit, multiply the current result by 10 and add the digit's value.
5. If it is not a digit, check if it is the null terminator (0). If not, set the result to -1 and exit.
6. If it is the null terminator, exit the loop.
7. Store the result in register `$v0`.

Here is the MIPS assembly code that performs the above steps:

```assembly
.data
    errorMsg: .asciiz "Non-digit character found.\n"

.text
.globl main
main:
    # Assume $a0 contains the address of the string
    li $v0, 0          # Initialize result to 0
    li $t1, -1         # Prepare -1 in $t1 to use if a non-digit is found

loop:
    lb $t0, 0($a0)     # Load byte (character) from string into $t0
    beq $t0, $zero, end_loop # If null terminator, end

In [ ]:
print(answer)